In [17]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import pandas.io.sql as psql
import kts
from kts import *
features.define_in_scope(globals())
helpers.define_in_scope(globals())

Using TensorFlow backend.


In [3]:
!ls

cnn-trainable-03-0.78.hdf5  homework-practice-01.ipynb	__pycache__
dost.py3		    indi.py			sentiment-analysis
dump.sql.tar.002	    indi.py3			sentimentModel.py
FeatureEngineering.ipynb    mystem			tokenizer.pickle


In [53]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql(f"SELECT uid FROM users WHERE data_loaded")

In [37]:
df = run_sql(f"SELECT uid FROM users WHERE first_name = 'Аня'")

In [38]:
df

""
uid
21754523
67223827
190775618
6608545
2124713
1175796
1162136
235314
4458494


In [112]:
df = run_sql(f"SELECT from_id, text FROM posts  WHERE (text <> '') AND (text LIKE '% да ладно тебе%')")

In [113]:
df

,from_id,text
0,12596704,ой да ладно тебе)) вот мне такое неестественно...
1,5537362,"ой, да ладно тебе РОм)"
2,661535,ахах))) любимый фильм- Scarface?))) да ладно ...
3,6261751,Ой да ладно тебе :) Без этого было бы гораздо ...
4,1739108,)) да ладно тебе!!))
5,841561,ой да ладно тебе) как поживаешь? когда в китай...
6,15865259,"ой, да ладно тебе))\nСама такая)))\nтебя я тож..."
7,5170843,хахаха да ладно тебе ))))
8,3949108,ой да ладно тебе=*
9,1639616,ой да ладно тебе)))


### Извлекаем эмодзи из сообщений

In [11]:
import emoji

In [12]:
def getEmojiFromString(text):
    listText = list(text)
    emojiList = [c for c in listText if c in emoji.UNICODE_EMOJI]
    return np.unique(emojiList, return_counts=True)

def getSarcasmEmojiFromString(text):
    sarcasmEmoji = ['🙃', '🐸', '💁', '🌝']
    emojiList = [c for c in text if c in emoji.UNICODE_EMOJI]
    sarcasmEmojiList = [c for c in emojiList if c in sarcasmEmoji]
    return np.unique(sarcasmEmojiList, return_counts=True)

def getNegativeEmojiFromString(text):
    negativeEmoji = ['😠', '😡', '😤', '👿', '👎', '👊', '👺', '🤬', '😈', '💀', '☠️', '🔫', '😕', '🙅', '😾', '🖕']
    emojiList = [c for c in text if c in emoji.UNICODE_EMOJI]
    sarcasmEmojiList = [c for c in emojiList if c in negativeEmoji]
    return np.unique(sarcasmEmojiList, return_counts=True)

def getNegativeEmojiValueFromString(s):
    negEmoji = getNegativeEmojiFromString(s)
    return negEmoji[1].sum()

### Выделяем сарказм из текста

In [13]:
import string 
def removePunctuationFromString(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def getSarcasmPhraseFromString(s):
    sarcasmVocabulary = ['ну да конечно', 'да что ты говоришь', 'да ладно', 'спасибо кэп', 'что бы мы без тебя делали',
                         'ага конечно', 'прям сразу', 'прямо сразу', 'ой конечно', 'ой ладно', 'да что ты', 'ой да конечно']
    strClean = removePunctuationFromString(s)
    sarcasmList = []
    for sarcasmExpression in sarcasmVocabulary:
        if sarcasmExpression in s:
            sarcasmList.append(sarcasmExpression)
    return sarcasmList

def getSarcasmValueFromString(s):
    sarcasmPhrase = getSarcasmPhraseFromString(s)
    sarcasmEmoji = getSarcasmEmojiFromString(s)
    return int(len(sarcasmPhrase) or len(sarcasmEmoji[0]))

In [54]:
s = 'ой да ладно прям стесняшка!))))'

In [55]:
removePunctuationFromString(s)

'ой да ладно прям стесняшка'

In [58]:
getSarcasmPhraseFromString(s)

['да ладно']

In [14]:
getSarcasmValueFromString(s)

1

## Sentiment Model


In [15]:
from sentimentModel import SentimentModel

In [18]:
model = SentimentModel()

In [21]:
model.predictText('Ты самая лучшая!!!')[0][0]

0.7289227

In [22]:
model.predictText('Ненавижу тебя!!!')[0][0]

0.036379784

## Hate index

In [26]:
def getPostsByUid(uid):
    sqlQuery = "SELECT from_id, text FROM posts  WHERE (text <> '') AND (from_id = " + str(uid) + ")"
    return run_sql(sqlQuery).text

In [107]:
def getHateVector(sentimentModel, uid):
    df = getPostsByUid(uid)
    if df.empty:
        return None
    sentimentValues = [sentimentModel.predictText(df[i])[0][0] for i in range(df.shape[0])]
    sarcasmValues = [getSarcasmValueFromString(df[i]) for i in range(df.shape[0])]
    negativeEmojiValue = [getNegativeEmojiValueFromString(df[i]) for i in range(df.shape[0])]
    
    statisticSentimnet = np.mean(sentimentValues)
    statisticSarcasm = np.mean(sarcasmValues)
    statisticEmoji = np.mean(negativeEmojiValue)
    
    NORM_VALUE = 0.88
    NORM_COEF = 0.05
    
    return min((1 - statisticSentimnet + NORM_COEF) * (1 + statisticSarcasm) * (1 + statisticEmoji) / NORM_VALUE, 1)

In [108]:
getHateVector(model, 58720818)

0.643421477236023

In [109]:
# использует негативные эмоджи
getHateVector(model, 230622560)

0.8634179440473231

In [110]:
# немного матерится
getHateVector(model, 14403306)

0.7856840978969227

In [111]:
getHateVector(model, 16048952)

0.6537100672721863